In [1]:
from flask import Flask, request, jsonify, Response, make_response
from flask_cors import CORS
import zipfile
from zipfile import ZipFile
import os
import json
import pandas as pd
import numpy as np

import time
import cv2
import base64

In [2]:
# 폴더안에있는 Json파일 전부
def jsonzip(od_path):
    with zipfile.ZipFile(od_path, 'r') as obj:
        obj.namelist()		# zip파일 내의 파일 이름을 가진 list 반환
        obj.extractall("C:/datazip/")


def dataframe(od_path) :
    temp = od_path
    path = temp.split(".")[0]
    filename = od_path.split("/")[2].split(".")[0]
    list = os.listdir(path)
    json_file_list = [path+'/'+ file for file in list if file.endswith('.json')]
    
    imagePath = [] #이미지 파일명 저장 리스트
    label=[] #분류 라벨 값 저장 리스트
    points_1_x=[] #boundary box x,y좌표 리스트
    points_1_y=[] #boundary box x,y좌표 리스트
    points_2_x=[] #boundary box x,y좌표 리스트
    points_2_y=[] #boundary box x,y좌표 리스트
    errorFile =[] #에러 파일 검출 리스트
    shapes_list = [] 
    imageHeight = []
    imageWidth = []
    ratio = []
    wide = []
    bwide = []

    for i in json_file_list: #json_file_list의 하나의 인덱스 마다 i라는 변수로 사용한다.
        with open(i,'r') as j: #
            try:
                contents=json.loads(j.read())  # open : r - 읽기모드, w-쓰기모드, a-추가모드
                for k in range(len(contents['shapes'])): # contents라는 json객체 안에 여러 객체가 있을 경우를 대비해 반복문안에 shapes의 길이로 반복한다
                                                        # shapes의 내용이 2개면 2번 반복하는 형태 --> 변수는 k사용
                    # print(contents['shapes'][k]) # 결과 확인 및 데이터 확인                 
                    shapes_list.append(len(contents['shapes']))
                    imagePath.append(str((contents['imagePath']).replace('.jpg',""))) # 따로 구별하는 변수가 없어서 이미지 경로에 확장자만 제거하여 사용
                    label.append(contents['shapes'][k]['label']) # label은 분리수거 되는 분류 클래스로 shapes->label 키 값으로 접근
                    points_1_x.append(contents['shapes'][k]['points'][0][0])
                    points_1_y.append(contents['shapes'][k]['points'][0][1])
                    points_2_x.append(contents['shapes'][k]['points'][1][0])
                    points_2_y.append(contents['shapes'][k]['points'][1][1])
                    imageHeight.append(contents['imageHeight'])
                    imageWidth.append(contents['imageWidth'])
            except:
                errorFile.append(i) # 크기가 0이거나, 파일 불러올 때 오류를 유발하는 파일명 errorFile리스트에 담는다.

    # 데이터 프레임 만들기
    # 딕셔너리 형태로 리스트를 묶어서 넣어준다.
    points_1_x=np.round(points_1_x).astype('int')
    points_1_y=np.round(points_1_y).astype('int')
    points_2_x=np.round(points_2_x).astype('int')
    points_2_y=np.round(points_2_y).astype('int')
    for i in range(len(imagePath)) :
        ratio.append((((points_2_x[i] - points_1_x[i])* (points_2_y[i] - points_1_y[i]))/( imageHeight[i] * imageWidth[i]))*100)
        wide.append(imageHeight[i]*imageWidth[i])
        bwide.append((points_2_x[i] - points_1_x[i])*(points_2_y[i] - points_1_y[i]))
        # print("Ratio for image", imagePath[i], ":", ratio)
        # print("Wide for image", imagePath[i], ":", wide[i])
        # print("Bide for image", imagePath[i], ":", bwide[i])

    ratio=np.round(ratio,0)

    data=pd.DataFrame({"imagePath":imagePath,
                        "label":label,
                        "Shape_num":shapes_list,
                        "point1_x":points_1_x,
                        "point1_y":points_1_y,
                        "point2_x":points_2_x,
                        "point2_y":points_2_y,
                        "imageHeight":imageHeight,
                        "imageWidth":imageWidth,
                        "ratio" : ratio,
                        "wide" : wide,
                        "bwide" : bwide
                        })
    
    data.to_csv(f"{path}/{filename}.csv",index=False)
    return f"{path}/{filename}.csv"

In [3]:
def ratio_cal(dx,dy,size):
    area = (dx*dy)/size
    x = int(np.sqrt(area*(dx/dy)).round())
    y = int(((dy/dx)*x).round())
    return x,y

In [4]:
def make_points(array, size):
    x1 = array[0][3]
    y1 = array[0][4]
    x2 = array[0][5]
    y2 = array[0][6]
    dx = np.abs(x2-x1)
    dy = np.abs(y2-y1)
    image = str(array[0])
    area = (dx*dy)/size
    x = int(np.sqrt(area*(dx/dy)).round())
    y = int(((dy/dx)*x).round())
    delta_x =int((x-dx)/2)
    delta_y =int((y-dy)/2)
    pt1_x = x1-delta_x
    pt1_y = y1-delta_y
    pt2_x = x2+delta_x
    pt2_y = y2+delta_y
    if pt1_x <0:
        pt1_x = x1-int(delta_x/2)
        pt2_x = x2+int(delta_x/2)
    if pt1_y <0:
        pt1_y = y1-int(delta_y/2)
        pt2_y = y2+int(delta_y/2)
    return pt1_x, pt1_y,pt2_x,pt2_y,image

In [5]:
def bgImage(filename,path,size):
    bg = int(size)/100
    data = pd.read_csv(f"C:/datazip/{path}/{path}.csv", index_col=False)
    file = filename.split(".")[0]
    array = data[data['imagePath']==int(file)].values
    
    x1,y1,x2,y2,image = make_points(array, bg)
    img = cv2.imread(f"C:/datazip/{path}/{filename}", cv2.IMREAD_COLOR)
    roi = img[y1:y2,x1:x2]
    cv2.imwrite(f"C:/datazip/{path}/{file}-{size}.jpg", roi)
    imagePath = f"{path}/{file}-{size}.jpg"
    return imagePath

In [6]:
def points(array):
    x1 = array[3]
    y1 = array[4]
    x2 = array[5]
    y2 = array[6]
    dx = np.abs(x2-x1)
    dy = np.abs(y2-y1)
    image = str(array[0])
    return x1,y1,x2,y2,dx,dy,image

In [32]:
def make_json(path, size):
    test_list = os.listdir("C:/datazip")
    file_temp = path+"-"+size 
    if (not file_temp in test_list):
        os.mkdir(f"C:/datazip/{path}-{size}")
    error_file=[]
    test = pd.read_csv(f'C:/datazip/{path}/{path}.csv', index_col=False)
    for i in range(len(test.values)):
        try:
            x1,y1,x2,y2,dx,dy,image = points(test.values[i])
            dx2, dy2 =ratio_cal(dx,dy,0.2)
            delta_x =int((dx2-dx)/2)
            delta_y =int((dy2-dy)/2)
            pt1_x = x1-delta_x
            pt1_y = y1-delta_y
            pt2_x = x2+delta_x
            pt2_y = y2+delta_y
            if pt1_x <0:
                pt1_x = x1-int(delta_x/2)
                pt2_x = x2+int(delta_x/2)
            if pt1_y <0:
                pt1_y = y1-int(delta_y/2)
                pt2_y = y2+int(delta_y/2)
            path2 = f'C:/datazip/{path}/' + image +".jpg"
            img = cv2.imread(path2, cv2.IMREAD_COLOR)
            roi = img[pt1_y:pt2_y,pt1_x:pt2_x]
            print((dx*dy)/(roi.shape[0]*roi.shape[1]))
            if ((dx*dy)/(roi.shape[0]*roi.shape[1])>=1):
                error_file.append(image)
            else:
                cv2.imwrite(f"C:/datazip/{path}-{size}/{image}-{size}.jpg", roi)
                dict = {
                    "version": "4.5.10",
                    "flags": {},
                    "shapes": [
                                {
                                    "label": test.values[i][1],
                                    "points": [
                                                [
                                                    x1-pt1_x,
                                                    y1-pt1_y
                                                ],
                                                [
                                                    x1-pt1_x+(x2-x1),
                                                    y1-pt1_y+(y2-y1)
                                                ]
                                    ],
                                    "group_id": "null",
                                    "shape_type": "rectangle",
                                    "flags": {}
                                    }
                                ],
                    "imagePath": f"{image}-{size}.jpg",
                    "imageData": "null",
                    "imageHeight": roi.shape[0],
                    "imageWidth": roi.shape[1]
            }
                with open(f"C:/datazip/{path}-{size}/{image}-{size}.json", 'w') as f:
                    json.dump(dict,f,indent=4)
                    
        except:
            error_file.append(image)
    os.chdir('C:/datazip')
    with zipfile.ZipFile(f'{path}-{size}.zip', "w") as zip_file: # "w": write 모드
        for file in os.listdir('C:/datazip/{path}-{size}'):
            zip_file.write(os.path.join("{path}-{size}",file))
        zip_file.close()
    return f"{path}-{size}.zip"

In [33]:
# 플라스크 실행 코드

app = Flask(__name__)
CORS(app)
# 클릭했을때 반환해주는거잖아
@app.route('/test', methods=['POST'])
def handle_ajax_request():
    time.sleep(1)
    od_path = request.form.get('od_path')  # Ajax 요청에서 'od_path' 값을 가져옵니다.
    print(od_path)
    path = od_path
    jsonzip(od_path)
    path = dataframe(od_path)
    # 응답을 반환합니다.

    data =pd.read_csv(path,index_col=False)
    # data['imagePath'] = data['imagePath'].astype("object")
    print(path)
    # response = data.to_json(date_format="cp949", orient='index', indent=4)
    # param = response.get_json()
    # return jsonify(data.to_dict())
    return Response(data.to_json(orient="records"), mimetype='application/json')

@app.route('/image', methods=['POST'])
def rest_img_test():
    if request.method == 'POST':
        fileName = request.form.get('imageName')
        filePath = request.form.get('imagePath')
        value = request.form.get('value')
        imagePath=bgImage(fileName,filePath,value)
        data = {'imagePath':imagePath}
        response = jsonify(data)
        
    return response
@app.route('/data', methods=['POST'])
def data():
    if request.method == 'POST':
        filePath = request.form.get('imagePath')
        value = request.form.get('value')
        zipPath = make_json(filePath, value)
        data = {'zipPath':zipPath}
        response = jsonify(data)
        
    return response

if __name__ == '__main__':
    app.run()
    

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


0.20000403901690328
0.20015560786817221
0.1999794372512075
0.20022464698331194
0.19990479873099007
0.2000842634955109
0.20000738522294628
0.2000290332949679
0.2000576808831047
0.2768284562817592
0.20030623042376505
0.20010461673902852
0.20025461601271324
0.1999990860383504
0.2000327171789958
0.20011743849912042
0.1999762620667827
0.20025669695385434
0.2001990259590481
0.20013802622498275
0.20004151526850097
0.20006655860997596
0.20030441400304413
0.19999570750971177
0.20004546743906698
0.19994628059144187
0.20001253997115806
0.200240619724633
0.20001593576868046
0.2005416437627825
0.19996219996219997
0.20018200222901605
0.2003818008272351
0.2767728325789205
0.27688397129186604
0.2000931808091383
0.20017023589831623
0.20009313144355986
0.20011646372770267
0.20019649771715886


[2023-06-12 18:38:59,814] ERROR in app: Exception on /data [POST]
Traceback (most recent call last):
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\